# Лабораторна робота №1

Бригада №3, КА-61м
- Одобеску Владислав
- Фатенко Владислав
- Шевченко Тарас

## Завдання

Вивчити теоретичні основи методу аналізу ієрархій.

Побудувати мультиплікативну матрицю парних порівнянь (МПП) оцінювання альтернатив рішень за одним критерієм за умови, що експерт дає оцінки у фундаментальній шкалі.

Розрахувати глобальні ваги елементів ієрархії для повної ієрархії з $p$ рівнями, $p>2$, яка задається вектором $m=\{(m_k)|k=1,\dots,p\}$, де $m_k \in \mathbb{N}$ - кількість елементів на $k$-му рівні ієрархії. Для цього:

- зчитати з файлу кількість рівнів ієрархії та кількість елементів на кожному рівні,
- зчитати з файлу матриці парних порівнянь елементів ієрархії,
- розрахувати один з показників узгодженості (згідно з варіантом) матриць парних порівнянь та зробити висновки щодо рівня неузгодженості,
- розрахувати локальні ваги елементів ієрархії одним з методів парних порівнянь (згідно з варіантом),
- розрахувати глобальні ваги елементів ієрархії одним з методів синтезу (згідно з варіантом).

Зробити висновки по роботі

Дати відповіді на контрольні питання, наведені в кінці роботи.

## Виконання роботи

### Побудова МПП
Розглядається задача №3:

> Нехай інвестор оцінює акції деякої компаній і хоче спрогнозувати, яким буде розподіл ймовірностей зміни ціни на них. Він розглядає наступні можливі варіанти зміни ціни: впаде на 20%, впаде на 10%, залишиться незмінною, зросте на 10%. Використовуючи результати фундаментального аналізу, парні порівняння варіантів зміни ціни наступні:
- імовірність події, що ціна акцій зросте на 10% ненабагато перевищує імовірність події, що ціна акцій залишиться незмінною на протязі визначеного періоду часу і суттєво перевищує імовірності того, що ціна акцій впаде як на 10%, так і на 20%;
- імовірність події, що ціна акцій залишиться незмінною ненабагато перевищує імовірності подій, що ціна акцій впаде як на 10%, так і на 20%;
- імовірність події, що ціна акцій впаде на 10% ненабагато перевищує імовірність події, що ціна акцій впаде на 20%.

Таблично умову можна представити у наступному вигляді:

<table>
<tr>
<th>Варіанти зміни ціни</th>
<th>Впаде на 20%</th>
<th>Впаде на 10%</th>
<th>Залишиться незмінною</th>
<th>Зросте на 10%</th>
</tr>
<tr>
<td>Впаде на 20%</td>
<td>-</td><td>-</td><td>-</td><td>-</td>
</tr>
<tr>
<td>Впаде на 10%</td>
<td>ненабагато перевищує</td><td>-</td><td>-</td><td>-</td>
</tr>
<tr>
<td>Залишиться незмінною</td>
<td>ненабагато перевищує</td><td>ненабагато перевищує</td><td>-</td><td>-</td>
</tr>
<tr>
<td>Зросте на 10%</td>
<td>суттєво перевищує</td><td>суттєво перевищує</td><td>ненабагато перевищує</td><td>-</td>
</tr>
</table>

Відповідно до фундаментальної шкали експертних суджень, "суттєво перевищує" відповідає 5, а "ненабагато перевищує" відповідає 3. Тоді отримуємо наступну МПП:

<table>
<tr>
<th>Варіанти зміни ціни</th>
<th>Впаде на 20%</th>
<th>Впаде на 10%</th>
<th>Залишиться незмінною</th>
<th>Зросте на 10%</th>
</tr>
<tr>
<td>Впаде на 20%</td>
<td>1</td><td>1/3</td><td>1/3</td><td>1/5</td>
</tr>
<tr>
<td>Впаде на 10%</td>
<td>3</td><td>1</td><td>1/3</td><td>1/5</td>
</tr>
<tr>
<td>Залишиться незмінною</td>
<td>3</td><td>3</td><td>1</td><td>1/3</td>
</tr>
<tr>
<td>Зросте на 10%</td>
<td>5</td><td>5</td><td>3</td><td>1</td>
</tr>
</table>

### Розрахування глобальних ваг елементів ієрархії

Розглядається варіант №3:
- $p=3$
- $m=\{2,3,4\}$
- метод головного власного вектору, показник узгодженості CR
- мультиплікативний метод розрахунку глобальних ваг

#### Імпорт бібліотек

In [1]:
%pylab inline
import seaborn as sns
import scipy as sp
import pandas as pd
from scipy import stats
from IPython.display import display

Populating the interactive namespace from numpy and matplotlib


### Підготовка програмних функцій

Для визначення локальних ваг альтернатив будемо використовувати EM-метод.

In [1]:
def cr_metric(lamb, dim):
    """
    Eats matrix of pairwise comparisons m and produces CR metric of consistensy.
    """
    # calculate ci metric
    assert lamb >= dim, "Greatest eigenvalue must be greater than matrix dimension."
    ci = (lamb - dim)
    if dim > 1:
        ci /= (dim - 1)
    # fill the MRCI table
    mrci = [0, 0, 0, 0.52, 0.89, 1.11, 1.25, 1.35, 1.40, 1.45, 1.49, 1.52, 1.54, 1.56, 1.58, 1.59] 
    cr = ci / mrci[m.shape[0]]
    return cr

def em_ci(mat):
    """
    Eats matrix of pairwise comparisons m and produces pair (local weights, CR metric)
    """
    assert mat.shape[0] == mat.shape[1]
    values, vectors = linalg.eig(mat)
    max_index = np.argmax(abs(values))
    v = vectors[:, max_index]
    lamb = values[max_index]
    assert (v.imag < 1e-6).all(), "eigenvector has complex coordinates: %s" % v.imag
    v = v.real
    if (v < 0).all():
        v = - v
    assert (v >= 0).all(), "eigenvector has negative coordinates: %s" % v
    return v / linalg.norm(v, ord=1), cr_metric(lamb, float(mat.shape[0]))

Нехай задані:
- множина альтернатив $A=\{a_i| \; i=1,\dots,n\}$;
- множина критеріїв (цілей) $C=\{C_j| \; j=1,\dots,m\}$;
- ваги критеріїв $W^C=\{w_j^C| \; j=1,\dots,m\}$;
- ваги альтернатив відносно критеріїв $W=\{w_{ij}| \; i=1,\dots,n; \; j=1,\dots,m\}$, $w_{ij}$ - вага $i$-ої альтернативи відносно $j$-ого критерію.

Необхідно визначити глобальні ваги альтернатив $W^{glob}=\{w_i^{glob}| \; i=1,\dots,n\}$.

Тоді процедура мультиплікативного синтезу визначається наступним чином:
$$w_{i} = \prod\limits_{j=1}^{m} (w_{ij})^{w_j^C}$$